<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=237856746" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from enum import Enum
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from threading import Timer
from tqdm import tqdm
from typing import Optional
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# A Gemini python api-helper with retry support.
class Gemini:
    gen_model = [("gemini-2.0-flash", 15),     # latest: 15 RPM / 1500 RPD / 500 search per day / 1M TPM
                 ("gemini-2.0-flash-001", 15), # stable: ...
                 ("gemini-2.0-flash-exp", 10), #    exp: 10 RPM / ...
                 ("gemini-2.5-flash-preview-04-17", 10), # 10 RPM / 500 RPD / 500 search per day / 250K TPM
                 ("gemini-2.5-pro-exp-03-25", 5)]        #  5 RPM /  25 RPD / 500 search per day / 250K TPM / 1M TPD
    embed_model = ("text-embedding-004", 1500) # 1500 RPM / Max 100 per batch embed request
    max_retry = 3
    min_rpm = 3
    dt_between = 2.0
    errored = False
    dt_reset = 60.0
    
    class Model(Enum):
        GEN = 1
        EMB = 2

    def __init__(self):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.timer = Timer(self.dt_reset, self.zero_error)
        self.gen_rpm = self.gen_model[0][1]
        self.m_id = 0

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + self.gen_model[self.m_id][0]
        else:
            return "models/" + self.embed_model[0]

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(self.max_retry):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error()
                return retry_fn(*args, **kwargs)
            except Exception as e:
                self.on_error()
                if attempt == self.max_retry-1:
                    raise e
                time.sleep(self.dt_between)

    def on_error(self):
        self.m_id = self.m_id+1 if self.m_id < len(self.gen_model)-1 else 0
        self.gen_rpm = self.gen_model[self.m_id][1]
        if not self.errored:
            self.timer.start()
            self.errored = True

    def zero_error(self):
        self.timer = Timer(self.dt_reset, self.zero_error)
        self.errored = False
        self.m_id = 0
        self.gen_rpm = self.gen_model[0][1]

# Create the api-helper.
api = Gemini()

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**General Information**
*   The stock market facilitates the exchange of stocks or securities between buyers and sellers. This transfer requires an agreement on price between the parties.
*   Equities, also known as stocks or shares, represent ownership in a company.
*   The stock market includes a range of participants, from individual investors to large institutions like banks, insurance companies, hedge funds, and pension funds. These participants can be located worldwide.
*   Orders to buy or sell are typically executed by stock exchange traders.

**Stock Exchanges**
*   A stock exchange is a marketplace where traders and stockbrokers can buy and sell shares, bonds, and other securities.
*   Many large companies have their stocks listed on a stock exchange, which increases the stock's liquidity and attractiveness to investors.
*   A stock exchange can also act as a guarantor of settlement.
*   Some companies list their stock on multiple exchanges in different countries to attract international investors.
*   Stocks can also be traded "over the counter" (OTC) through a dealer.

**Market Size**
*   The total market capitalization of all publicly traded stocks worldwide rose from US$2.5 trillion in 1980 to US$111 trillion by the end of 2023.
*   As of 2016, there were 60 stock exchanges in the world.
*   In 2021, the value of world stock markets experienced an increase of 26.5%, amounting to US$22.3 trillion.

**Primary and Secondary Markets**
*   In the primary market, financial assets are created and transmitted directly by their issuer.
*   In the secondary market, existing financial assets are exchanged.

Keep in mind that the stock market is dynamic and subject to change. For example, the United States Stock Market Index (US500) has decreased by 3.31% since the beginning of 2025.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Here's an overview of AMZN (Amazon) stock, incorporating recent data and analyst perspectives:

**Current Price and Performance**

*   As of May 3, 2025, the current price of AMZN is around $189.98.
*   In the past 24 hours, the price has decreased slightly.
*   Over the last week, AMZN stock has increased by 1.26%.
*   The stock has shown a 5.05% increase over the last year.
*   The 52-week high for AMZN is $242.52, reached on February 3, 2025, and the 52-week low is $151.61.

**Analyst Ratings and Price Targets**

*   The consensus rating for AMZN stock is "Strong Buy" or "Moderate Buy".
*   Based on ratings from 49 Wall Street analysts, the average 12-month price target for Amazon is $242.33.
*   Individual analyst price targets range from a low of $186 to a high of $290.
*   Bank of America Global Research recently increased its Amazon price target to $230 from $225, with a "Buy" rating.
*   Wedbush raised its price target on Amazon to $235 from $225, maintaining an "Outperform" rating.

**Earnings and Revenue**

*   Amazon's earnings for the last quarter were $1.59 per share, exceeding estimates.
*   Revenue for the last quarter amounted to $155.67 billion.
*   Analysts estimate earnings for the next quarter to be $1.32 per share.
*   In 2024, Amazon's revenue was $637.96 billion, a 10.99% increase compared to the previous year. Earnings were $59.25 billion, up 94.73%.

**Forecasts and Predictions**

*   Analysts predict revenue will grow by approximately 10% in 2025.
*   One source predicts that in 2025, AMZN is anticipated to trade between $180.44 and $212.40.
*   Another source forecasts a maximum price of $234 and a minimum of $175 for May 2025.
*   In a bullish scenario, Amazon's stock price could potentially double in five years.

**Key Statistics**

*   Amazon's market capitalization is approximately $2.02 trillion.
*   The price-to-earnings (P/E) ratio is around 30.97.
*   Amazon does not currently pay dividends.

**Factors to Consider**

*   **Tariffs:** Potential impacts of tariffs on Amazon's business are being closely monitored.
*   **AWS Growth:** While still a strong performer, growth in Amazon Web Services (AWS) has been a focus of analysis.
*   **Macroeconomic Conditions:** Overall economic uncertainty and potential downturns could affect Amazon's performance.
*   **Analyst Sentiment:** The majority of analysts have a positive outlook on AMZN stock.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's an overview of AMZN's current share price, short-term trends, and bullish versus bearish predictions:

**Current Share Price**

*   As of May 3, 2025, the current price of AMZN is around $189.98.
*   The price has decreased slightly, by -0.12%, in the past 24 hours.

**Short-Term Trends**

*   **Mixed Signals:** Technical indicators show a bearish sentiment, while the Fear & Greed Index indicates "Fear".
*   **Recent Performance:** AMZN stock recorded 17 green days out of the last 30, with 5.33% price volatility. It has risen by 1.26% compared to the previous week and 1.24% over the last month.
*   **Support and Resistance:** Immediate support lies near $178.41, with resistance at $190. The stock is testing resistance at $190, and an upward breakthrough could signal a positive trend.
*   **Falling Trend:** Given the current short-term trend, the stock is expected to fall -23.89% during the next 3 months.
*   **Tomorrow's Forecast:** Amazon is expected to reach the highest price of $189.98 tomorrow.
*   **Technical Analysis:** Amazon has broken the floor of the rising trend channel in the medium long term, which indicates a weaker initial rising rate.

**Bullish Predictions**

*   **Analyst Ratings:** The consensus rating for AMZN stock is "Strong Buy" or "Moderate Buy".
*   **Revenue Growth:** Analysts predict revenue will grow by approximately 10% in 2025.
*   **Price Targets:** The average 12-month price target from analysts is $242.33.
*   **Long-Term Potential:** In a bullish scenario, Amazon's stock price could potentially double in five years.
*   **Overall Growth:** In a normal economic environment, Amazon should be able to grow earnings roughly 20% annually over the next five years.
*   **Positive Factors:** Positive developments in AWS, along with a rebound in consumer spending, could drive the stock higher.
*   **Profit Potential:** If you invest $1,000 in Amazon today and hold until Jul 10, 2025, the prediction suggests a potential profit of $117.99, reflecting an 11.80% ROI over the next 68 days.

**Bearish Predictions**

*   **Short-Term Decline:** According to the current forecast, the value of Amazon shares is expected to drop by -3.29% and reach $183.73 per share by June 1, 2025.
*   **Economic Slowdown:** If the world economy stagnates, the 20% annual earnings gains will not materialize.
*   **Negative Factors:** Rising inflation, increased competition, or disappointing earnings reports could weigh on the stock.
*   **Price Channel:** In 2026, Amazon is forecasted to trade in a price channel between $94.39 and $193.27.
*   **Overall Bearish:** Generally speaking, Amazon price prediction for 2027 is bearish.
*   **Downside Risk:** Given the current short-term trend, the stock is expected to fall -23.89% during the next 3 months and, with a 90% probability hold a price between $120.75 and $144.59 at the end of this 3-month period.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

The stock ticker symbol for MGM Resorts International is MGM. It is listed on the New York Stock Exchange (NYSE).


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I can't directly execute Python code to generate a candlestick plot within this environment. However, I can provide you with the Python code using libraries like `matplotlib` or `plotly` that you can run in your own Python environment (like Jupyter Notebook, Google Colab, etc.) to create the candlestick chart.

Here's the Python code using `matplotlib`:

```python
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from mplfinance.original_flavor import candlestick_ohlc

# Data (replace with actual data)
data = {
    'Date': ['2025-04-28', '2025-04-29', '2025-04-30', '2025-05-01', '2025-05-02'],
    'Open': [32.21, 31.80, 31.46, 30.86, 31.49],
    'High': [32.21, 31.80, 31.46, 30.86, 32.30],
    'Low': [32.21, 31.80, 31.46, 30.86, 31.15],
    'Close': [32.21, 31.80, 31.46, 30.86, 31.97]
}

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].apply(mdates.date2num)

# Create the candlestick chart
fig, ax = plt.subplots()
candlestick_ohlc(ax, df[['Date', 'Open', 'High', 'Low', 'Close']].values, width=0.6, colorup='green', colordown='red')

# Format the x-axis
date_format = mdates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(date_format)
fig.autofmt_xdate()

# Add labels and title
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.set_title('MGM Candlestick Chart (Last Week)')

# Show the chart
plt.show()
```

**Explanation:**

1.  **Import Libraries:** Import `matplotlib.pyplot` for plotting, `matplotlib.dates` for handling dates, `pandas` for data manipulation, and `mplfinance` for candlestick chart functionality. If you don't have `mplfinance`, you can install it using `pip install mplfinance`.
2.  **Data:**  I've created a sample `data` dictionary.  **You MUST replace this with actual data** for the last week's open, high, low, and close prices for MGM.  I've used the data from the search results for demonstration.
3.  **Create DataFrame:** Convert the dictionary to a Pandas DataFrame.
4.  **Convert Dates:** Convert the 'Date' column to datetime objects and then to numerical dates that `matplotlib` can use.
5.  **Create Subplot:** Create a figure and an axes object.
6.  **Candlestick Chart:** Use `candlestick_ohlc` to create the candlestick chart.
    *   `ax`: The axes object to plot on.
    *   `df[['Date', 'Open', 'High', 'Low', 'Close']].values`:  The data in the correct format (Date, Open, High, Low, Close).
    *   `width`: Width of the candlesticks.
    *   `colorup`: Color for candlesticks where the closing price is higher than the opening price.
    *   `colordown`: Color for candlesticks where the closing price is lower than the opening price.
7.  **Format x-axis:**  Set the date format for the x-axis.
8.  **Labels and Title:** Add labels to the axes and a title to the chart.
9.  **Show Chart:** Display the chart.

**To use this code:**

1.  **Install Libraries:**  If you don't have them, install `pandas` and `mplfinance`:
    ```bash
    pip install pandas matplotlib mplfinance
    ```
2.  **Replace Sample Data:**  **Crucially, replace the sample data in the `data` dictionary with the *actual* open, high, low, and close prices for MGM for the last week.** You can get this data from a financial website like Yahoo Finance, Google Finance, or your brokerage account.
3.  **Run the Code:**  Run the Python code in your Python environment.  A candlestick chart will be displayed.

This will give you a basic candlestick chart. You can customize it further by adding volume bars, moving averages, and other technical indicators.


In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

Okay, I will generate Python code to plot the open, close, high, and low values as a candlestick chart. Since I don't have real-time access to stock data, I'll use placeholder data. You'll need to replace this placeholder data with actual data for MGM. I will use `plotly` to generate an interactive candlestick chart.

```python
import plotly.graph_objects as go
import pandas as pd

# Replace with actual MGM stock data
data = {
    'Date': ['2025-04-28', '2025-04-29', '2025-04-30', '2025-05-01', '2025-05-02', '2025-05-03'],
    'Open': [32.21, 31.80, 31.46, 30.86, 31.49, 32.00],
    'High': [32.50, 32.00, 31.75, 31.20, 32.30, 32.50],
    'Low': [31.90, 31.50, 31.20, 30.50, 31.15, 31.80],
    'Close': [31.95, 31.60, 31.00, 31.00, 31.97, 32.20]
}

df = pd.DataFrame(data)

fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.update_layout(
    title='MGM Stock Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False # Hide the range slider for cleaner look
)

fig.show()
```

Key improvements and explanations:

*   **Uses `plotly`:** This code uses `plotly`, which creates interactive charts that are much more user-friendly than `matplotlib` for this purpose.  You can zoom, pan, and hover over the candlesticks for detailed information.
*   **Clearer Data Structure:** The data is organized in a dictionary, making it easy to understand and modify.  It's then converted to a Pandas DataFrame, which `plotly` works well with.
*   **Complete Example:** This is a complete, runnable example.  Just copy and paste it into a Python environment with `plotly` installed.
*   **Date Handling:** The `plotly` library automatically handles the date formatting, so no explicit date conversion is needed.
*   **Layout Customization:** The `update_layout` function adds a title and axis labels, and hides the range slider for a cleaner look.  You can customize the layout further.
*   **Placeholder Data:**  Includes placeholder data.  **You *must* replace this with real data.**
*   **Conciseness:** The code is more concise and readable.

To use this code:

1.  **Install `plotly`:** If you don't have it, install it using pip:
    ```bash
    pip install plotly pandas
    ```
2.  **Replace Placeholder Data:** Replace the sample data in the `data` dictionary with the actual open, high, low, and close prices for MGM.
3.  **Run the Code:** Run the Python code. A candlestick chart will be displayed in your browser.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Here's the AMZN data for May 2, 2025:

*   **Open:** \$191.435
*   **High:** \$192.88
*   **Low:** \$186.40
*   **Close:** \$189.98


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here's the AMZN (Amazon) stock data for the past month (approximately April 3, 2025 - May 3, 2025).  All data is sourced from Nasdaq and Investing.com.  Note that data may be delayed.

| Date       | Open     | High     | Low      | Close    |
| ---------- | -------- | -------- | -------- | -------- |
| May 02, 2025 | $191.435 | $192.88  | $186.40  | $189.98  |
| May 01, 2025 | $190.63  | $191.8071| $187.50  | $190.20  |
| Apr 30, 2025 | $182.17  | $185.05  | $178.85  | $184.42  |
| Apr 29, 2025 | $183.99  | $188.0158| $183.68  | $187.39  |
| Apr 28, 2025 | $190.105 | $190.22  | $184.885 | $187.70  |
| Apr 25, 2025 | $187.62  | $189.94  | $185.49  | $188.99  |
| Apr 24, 2025 | $180.915 | $186.74  |  | $186.54  |
| Apr 23, 2025 | $183.45  | $187.38  |  | $180.60  |
| Apr 22, 2025 | $169.845 | $176.78  |  | $173.18  |
| Apr 21, 2025 | $169.85  | $176.78  | $169.35  | $173.18  |
| Apr 20, 2025 | $169.60  | $169.60  | $165.29  | $167.32  |
| Apr 16, 2025 | $176.00  | $176.21  | $172.00  | $172.61  |
| Apr 15, 2025 | $176.29  | $179.10  | $171.41  | $174.33  |
| Apr 14, 2025 | $181.41  | $182.35  | $177.93  | $179.59  |
| Apr 13, 2025 | $186.84  | $187.44  | $179.23  | $182.12  |
| Apr 10, 2025 | $179.93  | $185.86  | $178.00  | $184.87  |
| Apr 09, 2025 | $185.44  | $186.87  | $175.85  | $181.22  |
| Apr 08, 2025 | $172.12  | $192.65  | $169.93  | $191.10  |
| Apr 07, 2025 | $185.23  | $185.90  | $168.57  | $170.66  |
| Apr 06, 2025 | $162.00  | $183.41  | $161.38  | $175.26  |
| Apr 03, 2025 | $167.15  | $178.14  | $166.00  | $171.00  |
| Apr 02, 2025 | $182.99  | $184.13  | $176.92  | $178.41  |

Please note:

*   Gaps in the data may be present due to weekends or holidays when the stock market is closed.
*   Some low and high data may be missing from the source.
*   The data is delayed, so real-time prices may vary.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [11]:
# An embedding function based on text-embedding-004.
class GeminiEmbeddingFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), 100):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + 100])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts_between(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [12]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbeddingFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=2000) # chunk by token limit of models/text-embedding-004
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps({"question": query, "answer": doc.page_content}) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]})
            
    def add_wiki_documents(self, title: str, documents: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(documents))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(documents)
            tqdm(self.db.add(ids=ids, documents=documents, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.db.get(where={"source": "add_wiki_document"})
        else:
            return self.db.get(where={"title": title})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        result = self.db.query(query_texts=[query], n_results=max_sources, where=where)
        [all_passages] = result["documents"]
        [all_dist] = result["distances"]
        [all_meta] = result["metadatas"]
        return all_passages, all_dist, all_meta

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list] = None):
        passage_list, dist_list, meta_list = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        for passage in passage_list if passages is None else passage_list + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return api.retriable(self.client.models.generate_content, 
                             model=api(Gemini.Model.GEN), 
                             config=self.config_rag, 
                             contents=prompt)

In [13]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=2000, # chunk by token limit of models/text-embedding-004
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        result = self.rag.get_wiki_documents(topic)
        if len(result["documents"]) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, result["documents"]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.get_topic_similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        documents = [document.page_content for document in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, documents)
                        return self.rag.generate_with_wiki_passages(query, topic, documents).text

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_topic_similarity(self, topic: str, page: str):
        return GeminiEmbeddingFunction(api.client, semantic_mode = True).sts_between([topic + " company", page])

In [14]:
# An implementation of Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class GroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        docs = self.rag.get_grounding_documents(query, topic)
        if len(docs["documents"]) > 0:
            for i in range(len(docs["metadatas"])):
                doc = docs["documents"][i]
                meta_q = docs["metadatas"][i]["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.get_grounding_similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding_similarity(self, question: str, compare: str):
        return GeminiEmbeddingFunction(api.client, semantic_mode = True).sts_between([question, compare])

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return "I don't know." # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return "I don't know." # Empty grounding_supports means grounding not possible for query.

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [15]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = GroundingGenerator(api.client, tool_rag)

Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [16]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours. 
    Do not answer in full sentences. Omit all chat and provide the answer only. 
    All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri. Open/Close happens on weekdays. 
    All exchanges are closed on weekends. Weekends are: Sat, Sun. No Open/Close happens on weekends. 
    The fields pre_market and post_market both represent open hours.
    
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    When was the US exchange's last operating hours? Provide the last weekday's close. Include any post-market hours. 
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [17]:
# Declare callable functions using OpenAPI schema.
get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

get_local_datetime_1 = types.FunctionDeclaration(
    name="get_local_datetime_1",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. The default if omitted is 'US' for the 
                                  US exchanges. A dictionary mapping supported exchange codes (key) to their 
                                  description (value) can be obtained from get_exchange_codes_1. Search the values for
                                  a matching exchange code if unsure."""
            }
        },
        "required": ["exchange"]
    }
)

get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is one 
                                  weekday prior to get_last_market_close (excluding weekends). This date must never 
                                  be more recent than the default. Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'. The default
                                  value is one month ago."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from today's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "query", "adjusted", "sort", "limit"]
    },
)

get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Omit this parameter unless you're told told to filter by published_utc."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 100 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "order", "limit", "sort", "query"]
    }
)

get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [18]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def rag_exchange_codes_1(content):
    response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contaihttps://api.polygon.io/v3/reference/tickers/AAPL?apiKey=4xJe226Z23RZmEc1bN8az1zz4pmNWdOpn key:value pairs 
                                             mapping exchange code to name. Just the dictionary string.
                                             Omit all other information or details. Do not chat or use sentences.""")
    codes = list(ast.literal_eval(response.text.strip("\`")).items())
    return codes

def rag_exchange_code_1(content):
    codes = tool_rag.get_exchanges_csv(f"""What is the {content} exchange code? Return only the exchange codes 
                                           as a list in string form. Just the list string.
                                           Omit all other information or details. Do not chat or use sentences.""").text
    return ast.literal_eval(codes)

def rag_last_market_close(content):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    return tool_rag.get_exchanges_csv(
        f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def get_similarity_score(content):
    return GeminiEmbeddingFunction(api.client, semantic_mode = True).sts_between(content)
    
def impl_get_symbol_1(content, by_name: bool = True):
    response = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(response[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}"
        try:
            response = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            matches = []
            max_failed_match = len(response["result"]) if not by_name else 3
            p_desc_match = 0.80
            p_symb_match = 0.95
            if response["count"] > 0:
                for result in tqdm(response["result"], desc="Score similarity to query"):
                    if max_failed_match > 0:
                        desc = [content['q'].upper(), result["description"].split("-", -1)[0]]
                        symb = [content['q'].upper(), result["symbol"]]
                        if by_name and get_similarity_score(desc) > p_desc_match: 
                            matches.append(result["symbol"])
                        elif not by_name and get_similarity_score(symb) > p_symb_match:
                            matches.append(result["description"])
                            max_failed_match = 0
                        else:
                            max_failed_match -= 1
            if len(matches) > 0:
                tool_rag.add_api_document(content["query"], matches, content["q"], "get_symbol_1")
                return matches
            else:
                return "I don't know."
    else:
        doc = ast.literal_eval(response[0][0])[0]
        return doc["answer"]

def impl_get_name_1(content):
    return impl_get_symbol_1(content, by_name = False)

def impl_get_quote_1(content):
    quotes = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = dict(impl_get_market_status_1(content))["isOpen"]
    if len(quotes[0]) == 0 or isOpen: 
        return get_current_price_1(content)
    else:
        last_close = parse(rag_last_market_close(content)).timestamp()
        for quote in quotes[2]: # index [2] for metadata
            if quote["timestamp"] >= last_close:
                return quotes
        return get_current_price_1(content)

def get_current_price_1(content):
    url = f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    # This is a high-demand endpoint. Expect random failure under heavy (free) use.
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0 and response["t"] > 0:
            tool_rag.add_quote_document(content["query"], response, content["symbol"], response["t"], "get_quote_1")
            return list(response.items())
        return "I don't know."

def impl_get_market_status_1(content):
    url = f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}"
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0:
            return list(response.items())
        return "I don't know."

def impl_get_peers_1(content):
    docs = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(docs[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}"
        try:
            peers = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(peers) > 0:
                names = []
                for peer in peers:
                    if peer == content["symbol"]:
                        continue # skip including the query symbol in peers (included in metadata anyway)
                    name_lookup = dict(q=peer, exchange=content["exchange"], query=content["query"])
                    name = impl_get_name_1(name_lookup)
                    if name != "I don't know.":
                        p = {"symbol": peer, "name": name}
                        names.append(p)
                peers = {"symbol": content["symbol"], "peers": names}
                tool_rag.add_peers_document(content["query"], peers, content["symbol"], "get_peers_1", content['grouping'])
                return list(peers.items())
            return "I don't know."
    else:
        peers = ast.literal_eval(docs[0][0])[0]["answer"] # The first document should be most relevant.
        return list(peers.items())

def impl_local_datetime_1(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=est).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=est).strftime('%c')

def impl_get_financials_1(content):
    fins = tool_rag.get_api_documents(content["query"], content["symbol"], "get_financials_1")
    if len(fins[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}"
        try:
            fin = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if not fin:
                return "I don't know."
            tool_rag.add_api_document(content["query"], fin, content["symbol"], "get_financials_1")
            return list(fin.items())
    return fins

def impl_get_news_1(content):
    #news = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    #if len(news[0]) == 0:
        url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
        try:
            news = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(news) == 0:
                return "I don't know."
            #tool_rag.add_api_document(content["query"], news, content["symbol"], "get_news_1")
            return list(news.items())
    #return news

def impl_daily_candle_2(content):
    daily_candle = tool_rag.get_api_documents(content["query"], content["stocksTicker"], "daily_candle_2")
    if len(daily_candle[0]) == 0:
        url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            daily_candle = ast.literal_eval(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if daily_candle["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], daily_candle, content["stocksTicker"], "daily_candle_2")
                return list(daily_candle.items())
            else:
                date = parse(content["date"])
                new_date = None
                if date.weekday() == 4: # index 4 for friday
                    new_date = date - timedelta(days=1)
                elif date.weekday() == 0: # index 0 for monday
                    new_date = date - timedelta(days=3)
                if new_date is None:
                    return "I don't know."
                content["date"] = new_date.strftime("%Y-%m-%d")
                return impl_daily_candle_2(content)
    return daily_candle

def impl_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        request = requests.get(url)
        custom_candle = json.loads(request.text)
    except:
        return f"I don't know. Endpoint returned status {request.status_code}"
    else:
        if custom_candle["status"] in ["OK","DELAYED"]:
            tool_rag.add_api_document(content["query"], custom_candle, content["stocksTicker"], "custom_candle_2")
            return list(custom_candle.items())
        return "I don't know."

def impl_ticker_overview_2(content):
    overview = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(overview[0]) == 0:
        url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            overview = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if overview["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], overview, content["ticker"], "ticker_overview_2")
                return list(overview.items())
            return "I don't know."
    return overview

def impl_trends_1(content):
    trends = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(trends[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
        try:
            trends = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(trends) > 0:
                tool_rag.add_api_document(content["query"], trends, content["symbol"], "trends_1")
                return trends
            return "I don't know."
    return trends

def impl_get_news_2(content):
    #news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    #if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                #tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return "I don't know."
    #return news
        
finance_tool = types.Tool(
    function_declarations=[
        get_symbol_1,
        get_name_1,
        get_symbol_quote_1,
        get_market_status_1,
        get_company_peers_1,
        get_local_datetime_1,
        get_last_market_close,
        get_exchange_codes_1,
        get_exchange_code_1,
        get_financials_1,
        get_daily_candlestick_2,
        get_custom_candlestick_2,
        get_ticker_overview_2,
        get_recommendation_trends_1,
        get_news_with_sentiment_2,
        get_rag_tool_response,
        get_wiki_tool_response,
        get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": impl_get_symbol_1,
    "get_name_1": impl_get_name_1,
    "get_symbol_quote_1": impl_get_quote_1,
    "get_market_status_1": impl_get_market_status_1,
    "get_company_peers_1": impl_get_peers_1,
    "get_local_datetime_1": impl_local_datetime_1,
    "get_last_market_close": rag_last_market_close,
    "get_exchange_codes_1": rag_exchange_codes_1,
    "get_exchange_code_1": rag_exchange_code_1,
    "get_financials_1": impl_get_financials_1,
    "get_daily_candlestick_2": impl_daily_candle_2,
    "get_custom_candlestick_2": impl_custom_candle_2,
    "get_ticker_overview_2": impl_ticker_overview_2,
    "get_recommendation_trends_1": impl_trends_1,
    "get_news_with_sentiment_2": impl_get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [19]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime_1 and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

In [20]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [21]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Gemini's innate notion of current date and time is unstable.
    est = pytz.timezone('US/Eastern') # The finance api data is in eastern time.
    contents += f"""
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = api.retriable(api.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=config_fncall, 
                                 contents=contents)
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [22]:
send_message("What is the current price of Amazon stock?")

Generate quote embedding: 0it [00:00, ?it/s]


#### Natural language response

The current price of Amazon (AMZN) stock is \\$189.98. The price changed by -\\$0.22, which is -0.1157% since the previous close. The high price of the day was \\$192.88, and the low was \\$186.4. The opening price for the day was \\$191.435, and the previous close price was \\$190.2. The price was last updated on Saturday, May 3, 2025 at 4:00 PM.


In [23]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.""")

Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

As of May 4, 2025, at 17:26:56, the current share price of Amazon (AMZN) is \\$189.98.

Here is the candlestick data for the past month (April 4, 2025, to May 2, 2025), sorted in descending order by date:

| Date            | Open    | High    | Low     | Close   | Volume      |
| :-------------- | :------ | :------ | :------ | :------ | :---------- |
| May 02, 2025    | \\$191.44 | \\$192.88 | \\$186.40 | \\$189.98 | 77,677,487  |
| May 01, 2025    | \\$190.63 | \\$191.81 | \\$187.50 | \\$190.20 | 74,228,963  |
| April 30, 2025  | \\$182.17 | \\$185.05 | \\$178.85 | \\$184.42 | 55,176,543  |
| April 29, 2025  | \\$183.99 | \\$188.02 | \\$183.68 | \\$187.39 | 41,667,255  |
| April 28, 2025  | \\$190.11 | \\$190.22 | \\$184.89 | \\$187.70 | 33,224,732  |
| April 25, 2025  | \\$187.62 | \\$189.94 | \\$185.49 | \\$188.99 | 36,413,330  |
| April 24, 2025  | \\$180.92 | \\$186.74 | \\$180.18 | \\$186.54 | 43,051,696  |
| April 23, 2025  | \\$183.45 | \\$187.38 | \\$180.19 | \\$180.60 | 63,470,094  |
| April 22, 2025  | \\$169.85 | \\$176.78 | \\$169.35 | \\$173.18 | 56,607,202  |
| April 21, 2025  | \\$169.60 | \\$169.60 | \\$165.29 | \\$167.32 | 48,126,111  |
| April 17, 2025  | \\$176.00 | \\$176.21 | \\$172.00 | \\$172.61 | 44,726,453  |
| April 16, 2025  | \\$176.29 | \\$179.10 | \\$171.41 | \\$174.33 | 51,866,916  |
| April 15, 2025  | \\$181.41 | \\$182.35 | \\$177.93 | \\$179.59 | 43,617,902  |
| April 14, 2025  | \\$186.84 | \\$187.44 | \\$179.23 | \\$182.12 | 48,002,540  |
| April 11, 2025  | \\$179.93 | \\$185.86 | \\$178.00 | \\$184.87 | 50,594,339  |
| April 10, 2025  | \\$185.44 | \\$186.87 | \\$175.85 | \\$181.22 | 68,302,045  |
| April 09, 2025  | \\$172.12 | \\$192.65 | \\$169.93 | \\$191.10 | 116,804,328 |
| April 08, 2025  | \\$185.23 | \\$185.90 | \\$168.57 | \\$170.66 | 87,710,360  |
| April 07, 2025  | \\$162.00 | \\$183.41 | \\$161.38 | \\$175.26 | 109,297,115 |
| April 04, 2025  | \\$167.15 | \\$178.14 | \\$166.00 | \\$171.00 | 123,136,859 |

The table provides a detailed view of Amazon's stock performance over the specified period, including the opening, high, low, and closing prices, as well as the trading volume for each day.


In [24]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

As of May 4, 2025, here's a summary of the bullish versus bearish predictions and recommendation trends for Amazon, incorporating sentiment analysis and short-term trends from recent news:

**Recommendation Trends:**

*   Based on analyst recommendations, there is a strong bullish sentiment towards Amazon. In May 2025, there were 22 strong buy, and 51 buy recommendations, 6 hold recommendations, and no sell or strong sell recommendations. This trend has remained relatively consistent over the past few months.

**Sentiment Analysis:**

*   Recent news articles reflect a generally positive sentiment toward Amazon, particularly highlighting its strong position in cloud computing (Amazon Web Services), its investments in artificial intelligence, and its resilient retail business.
*   Several articles suggest that Amazon is well-positioned for long-term growth, with analysts seeing potential for the stock to rally.
*   However, there are concerns about the potential impact of tariffs on Amazon's e-commerce business, as well as broader macroeconomic headwinds. Some analysts have downgraded the stock and lowered the price target due to these concerns.
*   Some articles suggest that Amazon is facing increasing competition in the cloud AI war with Microsoft.

**Short-Term Trends:**

*   The stock market sell-off has created a buying opportunity for Amazon, as its valuation is currently attractive.
*   The potential easing of trade tensions between the U.S. and China could provide a boost to Amazon's business.
*   Upcoming earnings reports from Amazon and other major tech companies will be closely watched by investors.

**Overall Summary:**

The overall sentiment towards Amazon is bullish, with analysts and investors recognizing the company's strong position in key growth markets like cloud computing and AI. While there are some near-term challenges related to tariffs and the economy, Amazon's long-term growth prospects remain strong.


In [26]:
send_message("What is Google's stock ticker symbol?")

Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 13.96it/s]


#### Natural language response

Google's stock ticker symbols on the NASDAQ are GOOGL and GOOG. It is also listed on the Frankfurt Stock Exchange under the ticker symbol GGQ1. These ticker symbols refer to Alphabet Inc., Google's holding company.


In [27]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

Score similarity to query: 100%|██████████| 11/11 [00:02<00:00,  4.43it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Google's (Alphabet Inc.) stock performance and news sentiment over the past month:

**Stock Performance (using GOOGL ticker):**

Over the past month, from April 4, 2025, to May 2, 2025, Google's share price (class A shares) experienced some volatility.

*   The share price started at \\$145.60 on April 4, 2025.
*   It reached a high of \\$166.10 on April 25, 2025.
*   It closed at \\$164.03 on May 2, 2025.

**News Sentiment:**

The news sentiment surrounding Google (Alphabet) over the past month has been generally positive, with several articles highlighting the company's strong earnings, growth potential in artificial intelligence, and attractive valuation. However, there are also some concerns about potential headwinds, such as increased spending, competition, and antitrust lawsuits.

*   **Positive Sentiment:** Several articles suggest that Alphabet is a good investment opportunity due to its strong earnings results, growth in artificial intelligence, and attractive valuation. Some analysts believe the stock is undervalued and expect it to climb in the coming months. The company's \\$70 billion share repurchase program and dividend hike are also seen as positive factors.
*   **Neutral Sentiment:** Some articles present a balanced view of Alphabet, acknowledging both its strengths and weaknesses. These articles mention potential headwinds such as trade wars, legal challenges, and competition from other tech companies.
*   **Negative Sentiment:** Some articles express concerns about the impact of tariffs, potential antitrust lawsuits, and increased spending on Alphabet's stock price. There are also concerns about the sustainability of Google Search and the company's ability to compete with other AI upstarts.

**Key Themes from the News:**

*   **Artificial Intelligence:** Alphabet is heavily investing in artificial intelligence, which is driving growth in its cloud, search, and advertising businesses. The company is also developing new AI solutions and partnering with other companies to expand its AI capabilities.
*   **Valuation:** Several articles suggest that Alphabet's stock is undervalued, making it an attractive investment opportunity. The company's low forward price-to-earnings ratio and strong earnings growth potential are seen as positive factors.
*   **Competition:** Alphabet faces increasing competition from other tech companies in areas such as search, cloud computing, and artificial intelligence. The company is also facing antitrust lawsuits from the U.S. Justice Department.
*   **Tariffs:** The potential impact of tariffs on Alphabet's business is a concern for some investors. However, the company's geographic diversification and strong fundamentals are expected to help it weather any potential headwinds.

**Disclaimer:** I am an AI chatbot and cannot provide financial advice. This information is for informational purposes only.


In [28]:
send_message("What is MGM Studio's stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

I was unable to find the stock symbol for Mgm Studios.

In [29]:
send_message("What is Amazon MGM Studio's stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]
Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

Amazon Mgm Studios is a subsidiary of Amazon, and its stock is under the stock symbol AMZN on the NASDAQ exchange.


In [30]:
send_message("What is Facebook's stock ticker symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]
Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

The stock ticker symbol for Facebook, now known as Meta Platforms Inc., is META. It is listed on the NASDAQ. Facebook's original ticker symbol was FB, which it used when it first went public. The company changed its name to Meta in 2021 and subsequently changed its ticker symbol to META.


In [31]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on Apple's peers. Then compare Apple result to it's peers.''')

Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 78.63it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 77.89it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 78.83it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 83.09it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 78.18it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 81.37it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 82.40it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 74.20it/s]
Generate api embedding: 0it [00:00, ?it/s]
Generate peer

#### Natural language response

The outlook for Apple (AAPL) is mixed based on recent analyst trends and news sentiment.

**Analyst Recommendations:**
Over the past few months, analyst recommendations for Apple have remained relatively stable, with a consistent number of "Buy" and "Strong Buy" ratings outweighing "Hold" and "Sell" ratings. In May 2025, there were 25 "Buy," 14 "Hold," and 3 "Sell" ratings, along with 14 "Strong Buy" and 0 "Strong Sell" ratings.

**News Sentiment:**
News sentiment for Apple is also mixed. Some articles highlight positive aspects, such as the company's strong Q2 results, cash generation, and potential for growth in a difficult environment. Other articles point to challenges, including tariff impacts, weaker sales in China, and regulatory headwinds.

**Peer Analysis:**
To provide a comparative perspective, an analysis of Apple's peers was conducted. The peers identified were Dell Technologies (DELL), HP Inc (HPQ), Hewlett Packard Enterprise (HPE), Super Micro Computer Inc (SMCI), NetApp Inc (NTAP), Pure Storage Inc (PSTG), Western Digital Corp (WDC), and IonQ Inc (IONQ).

**Dell Technologies (DELL):**
Analyst recommendations for Dell Technologies are generally positive, with a majority of "Buy" ratings. Recent news sentiment is also favorable, with many articles highlighting the company's potential to benefit from the AI surge and its strong position in the AI server market.

**HP Inc (HPQ):**
Analyst recommendations for HP Inc. are more neutral, with a higher number of "Hold" ratings compared to "Buy" ratings. Recent news sentiment is mixed, with some articles highlighting positive aspects such as the HP All-In Plan and collaborations with Scuderia Ferrari, while others point to challenges such as tariff risks and declining earnings quality.

**Summary:**

*   **Apple (AAPL):** The outlook is cautiously optimistic, with positive analyst recommendations but mixed news sentiment due to challenges in China and tariff concerns.
*   **Dell Technologies (DELL):** The outlook is positive, with favorable analyst recommendations and news sentiment driven by the company's potential to benefit from the AI surge.
*   **HP Inc (HPQ):** The outlook is neutral, with mixed analyst recommendations and news sentiment reflecting both positive initiatives and potential challenges.

In conclusion, while Apple faces some headwinds, its strong brand and potential for growth in new areas like AI suggest a positive long-term outlook. Dell Technologies appears well-positioned to benefit from the AI surge, while HP Inc. faces a more uncertain outlook due to mixed analyst sentiment and potential challenges in its core businesses.

The current date and time is May 4, 2025, at 17:29:38.

In [33]:
send_message('''What does the recent news say about Apple and the impact of tariffs? Over the past 2 month.''')

#### Natural language response

Recent news regarding Apple and the impact of tariffs over the past two months indicates a complex and evolving situation. Here's a summary:

**Tariff Impact and Mitigation:**

*   **Financial Impact:** Apple's fiscal Quarter 2 earnings were impacted by tariffs, and the company anticipates a \\$900 million increase in tariff-related costs in Quarter 3.
*   **Mitigation Strategies:** Apple has been actively working to lessen the impact of tariffs by building up inventory and shifting production to countries like India and Vietnam to avoid higher tariffs on Chinese imports.
*   **Production Shifts:** Apple is shifting more iPhone production to India to mitigate the impact of tariffs.

**Analyst Perspectives:**

*   **Price Target Cuts:** Bank of America cut Apple's price target to \\$240 due to tariff risks and delayed artificial intelligence rollout for future iPhones.
*   **Uncertain Outlook:** Apple faces an uncertain outlook due to tariff concerns and weaker sales in China, despite better-than-expected earnings per share and revenue in Quarter 2.

**Overall Sentiment:**

*   **Mixed Sentiment:** While Apple has taken steps to mitigate the effects of tariffs, the future impact remains unclear. The company faces challenges in China due to trade tensions.
*   **Long-Term Concerns:** There are concerns about Apple's earnings quality and reliance on share buybacks to maintain performance.

**Other Factors:**

*   **Trade Talk Considerations:** China is considering starting trade talks with the U.S., which could ease tensions.
*   **Temporary Tariff Exemption:** Apple received a temporary exemption from tariffs on its electronics, providing some relief.
*   **Tariff-Driven Demand:** Short-term sales benefit from tariff-driven demand pull-forward, but supply chain costs threaten 2026 margins.

In summary, Apple is navigating a challenging environment with tariff pressures and geopolitical tensions. While the company is taking steps to mitigate the impact, the long-term effects remain uncertain.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>